In [1]:
import numpy as np
import pandas as pd
from glob import glob


In [2]:
# config

pft = 1 # 何時間後のDSTを予測するか
udt = 6 # (udt-1)時間前までのデータを利用するか。つまり、Dstはudt個、OMNIデータは、udt*60/(1 or 5)個のデータ使う
omni_step_min = 5 # OMNIデータの時間間隔(minutes)


In [3]:
# データ整備
dst_data_path  = ''
omni_data_path = ''

# データロード, numpy 変換
train_data_dir      = '/Volumes/data/swfpy/ml/storm_datasets/Gruet_2018/train/'
train_dst_data_dir  = train_data_dir + 'DST/'
train_omni_data_dir = train_data_dir + 'omni_5min/'

test_data_dir      = '/Volumes/data/swfpy/ml/storm_datasets/Gruet_2018/test/'
test_dst_data_dir  = test_data_dir + 'DST/'
test_omni_data_dir = test_data_dir + 'omni_5min/'



In [5]:
class Dataset():
    
    def __init__(self, udt, omni_size):
        self.dst_f   = np.empty((0, 1))
        self.dst_p   = np.empty((0, udt))
        self.omni_bz = np.empty((0, omni_size))
        self.omni_b  = np.empty((0, omni_size))
        self.omni_v  = np.empty((0, omni_size))
        self.omni_n  = np.empty((0, omni_size))
    
    def create_dataset(self, dst_data_dir, omni_data_dir):
        dst_data_paths  = sorted(glob(dst_data_dir + '*'))
        omni_data_paths = sorted(glob(omni_data_dir + '*'))

        for dst_data_path, omni_data_path in zip(dst_data_paths, omni_data_paths):
            self._append_dataset(dst_data_path, omni_data_path)
    
    def _append_dataset(self, dst_data_path, omni_data_path):
#         print(dst_data_path, omni_data_path)
        dst_df  = pd.read_csv(dst_data_path)
        omni_df = pd.read_csv(omni_data_path)

        
        dst_rows = len(dst_df)
#         print(dst_df)
        for i in range(dst_rows - (udt + pft - 1)):

            dst_step = i + udt
            dst_f = np.double(dst_df[dst_step + (pft - 1):dst_step + 1 + (pft - 1)]['DST[nT]'])
            self.dst_f = np.append(self.dst_f, np.array([[dst_f]]), axis=0)

            dst_p = dst_df[i:dst_step]['DST[nT]'].values
#             print(dst_p)
            self.dst_p = np.append(self.dst_p, np.array([dst_p]), axis=0)

            # OMNIデータ準備
            omni_range_start = i*int(60/omni_step_min)
            omni_range_end   = i*int(60/omni_step_min) + (udt-1)*int(60/omni_step_min) + 1
            this_df = omni_df[omni_range_start:omni_range_end]
            bz    = this_df['BZ_GSM'].values
            b     = this_df['F'].values
            dens  = this_df['proton_density'].values
            speed = this_df['flow_speed'].values

            self.omni_bz = np.append(self.omni_bz, np.array([bz]), axis=0)
            self.omni_b  = np.append(self.omni_b, np.array([b]), axis=0)
            self.omni_v  = np.append(self.omni_v, np.array([dens]), axis=0)
            self.omni_n  = np.append(self.omni_n, np.array([speed]), axis=0)



In [6]:
omni_size   = (udt-1)*int(60/omni_step_min)+1
train_ds = Dataset(udt, omni_size)
train_ds.create_dataset(train_dst_data_dir, train_omni_data_dir)


In [6]:
train_ds.omni_n.shape

(4615, 61)

In [7]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Flatten, Dropout, Lambda, concatenate
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam


# inputはdst_p, bz, b, n, v
# shapeは、

dst_p_input = Input(shape=(udt,))
bz_input    = Input(shape=(omni_size,))
b_input     = Input(shape=(omni_size,))
n_input     = Input(shape=(omni_size,))
v_input     = Input(shape=(omni_size,))


dst_l = Dense(udt, activation='relu')(dst_p_input)
bz_l  = Dense(omni_size, activation='relu')(bz_input)
b_l   = Dense(omni_size, activation='relu')(b_input)
n_l   = Dense(omni_size, activation='relu')(n_input)
v_l   = Dense(omni_size, activation='relu')(v_input)

x = concatenate([dst_l, bz_l, b_l, n_l, v_l])
x = Dense(100, activation='relu')(x)
x = Dense(10, activation='relu')(x)
out = Dense(1, activation='linear')(x)

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model = Model(inputs=[dst_p_input, bz_input, b_input, n_input, v_input], outputs=out)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])


Using TensorFlow backend.
W0727 15:28:49.262034 139755790980864 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 15:28:49.274471 139755790980864 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 15:28:49.279401 139755790980864 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 15:28:49.364139 139755790980864 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/optimizers.py:

In [8]:
epochs = 100
batch_size = 100

mini = 1
model.fit(
#     [train_ds.dst_p[:mini], train_ds.omni_bz[:mini], train_ds.omni_b[:mini], train_ds.omni_v[:mini], train_ds.omni_n[:mini]],
    [train_ds.dst_p, train_ds.omni_bz, train_ds.omni_b, train_ds.omni_v, train_ds.omni_n],
    train_ds.dst_f,
#     train_ds.dst_f[:mini],
    batch_size=batch_size,
    epochs=epochs,
)




W0727 15:28:49.488419 139755790980864 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0727 15:28:49.625773 139755790980864 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/100
4615/4615 [==============================] - 3s 756us/step - loss: 3186.7782 - mean_absolute_error: 44.4870
Epoch 2/100
4615/4615 [==============================] - 0s 29us/step - loss: 376.2058 - mean_absolute_error: 12.0956
Epoch 3/100
4615/4615 [==============================] - 0s 28us/step - loss: 131.1070 - mean_absolute_error: 6.5505
Epoch 4/100
4615/4615 [==============================] - 0s 28us/step - loss: 100.8130 - mean_absolute_error: 5.9124
Epoch 5/100
4615/4615 [==============================] - 0s 29us/step - loss: 92.9844 - mean_absolute_error: 5.6886
Epoch 6/100
4615/4615 [==============================] - 0s 28us/step - loss: 91.7564 - mean_absolute_error: 5.7085
Epoch 7/100
4615/4615 [==============================] - 0s 29us/step - loss: 88.0429 - mean_absolute_error: 5.5704
Epoch 8/100
4615/4615 [==============================] - 0s 28us/step - loss: 85.8531 - mean_absolute_error: 5.4743
Epoch 9/100
4615/4615 [==============================] - 0s 29us

4615/4615 [==============================] - 0s 31us/step - loss: 72.4169 - mean_absolute_error: 5.1869
Epoch 72/100
4615/4615 [==============================] - 0s 28us/step - loss: 72.5802 - mean_absolute_error: 5.2098
Epoch 73/100
4615/4615 [==============================] - 0s 29us/step - loss: 72.3481 - mean_absolute_error: 5.1882
Epoch 74/100
4615/4615 [==============================] - 0s 33us/step - loss: 72.0872 - mean_absolute_error: 5.2196
Epoch 75/100
4615/4615 [==============================] - 0s 28us/step - loss: 74.9415 - mean_absolute_error: 5.3268
Epoch 76/100
4615/4615 [==============================] - 0s 30us/step - loss: 71.8218 - mean_absolute_error: 5.1571
Epoch 77/100
4615/4615 [==============================] - 0s 30us/step - loss: 71.3070 - mean_absolute_error: 5.1158
Epoch 78/100
4615/4615 [==============================] - 0s 30us/step - loss: 72.5664 - mean_absolute_error: 5.1763
Epoch 79/100
4615/4615 [==============================] - 0s 30us/step - loss

In [10]:
# モデルの保存
from keras.utils.vis_utils import plot_model

models_dir      = '/home/hirotoshi/PycharmProjects/swfpy/playground/models/'
model_plots_dir = '/home/hirotoshi/PycharmProjects/swfpy/playground/model_plots/'
model_out  = models_dir + 'd100_d10_model.h5'
model_plot = model_plots_dir + 'd100_d10_model_plot.svg'

model.save(model_out)
plot_model(model, to_file=model_plot, show_shapes=True)



ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [7]:
omni_size   = (udt-1)*int(60/omni_step_min)+1
test_ds = Dataset(udt, omni_size)
test_ds.create_dataset(test_dst_data_dir, test_omni_data_dir)

In [12]:
# モデルのロード
from keras.models import load_model
models_dir      = '/home/hirotoshi/PycharmProjects/swfpy/playground/models/'
model_plots_dir = '/home/hirotoshi/PycharmProjects/swfpy/playground/model_plots/'
model_out  = models_dir + 'd100_d10_model.h5'
model_plot = model_plots_dir + 'd100_d10_model_plot.svg'

model=load_model(model_out)





Using TensorFlow backend.
W0727 15:49:23.824486 140437454079744 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 15:49:23.837243 140437454079744 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 15:49:23.927916 140437454079744 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0727 15:49:23.928506 140437454079744 deprecation_wrapper.py:119] From /home/hirotoshi/PycharmProjects/swfpy/venv/lib/python3.7/site-packages/keras/backend/t

In [18]:
model.predict([test_ds.dst_p[:40], test_ds.omni_bz[:40], test_ds.omni_b[:40],
               test_ds.omni_v[:40], test_ds.omni_v[:40]], batch_size=10)

array([[ -20.030476],
       [ -25.99892 ],
       [ -28.925468],
       [ -26.35897 ],
       [ -26.441568],
       [ -23.601086],
       [ -47.282776],
       [ -70.297356],
       [ -65.92705 ],
       [ -68.97322 ],
       [ -80.78033 ],
       [ -77.671196],
       [-100.535034],
       [ -84.59345 ],
       [ -92.01246 ],
       [-104.72962 ],
       [ -87.65472 ],
       [ -82.67848 ],
       [ -73.71185 ],
       [ -82.39667 ],
       [ -74.03256 ],
       [ -83.0959  ],
       [ -72.18425 ],
       [ -71.59459 ],
       [ -62.892986],
       [ -62.770126],
       [ -58.170834],
       [ -50.08724 ],
       [ -42.15145 ],
       [ -53.99006 ],
       [ -46.787148],
       [ -49.33923 ],
       [ -31.96058 ],
       [ -24.894058],
       [ -22.61734 ],
       [ -34.556004],
       [ -93.19702 ],
       [-100.57233 ],
       [ -84.617516],
       [-127.463844]], dtype=float32)

In [19]:
test_ds.dst_f[:40]

array([[ -23.],
       [ -27.],
       [ -26.],
       [ -26.],
       [ -24.],
       [ -39.],
       [ -61.],
       [ -64.],
       [ -67.],
       [ -78.],
       [ -80.],
       [ -97.],
       [ -90.],
       [ -93.],
       [-105.],
       [ -94.],
       [ -86.],
       [ -79.],
       [ -84.],
       [ -78.],
       [ -83.],
       [ -77.],
       [ -74.],
       [ -67.],
       [ -65.],
       [ -61.],
       [ -53.],
       [ -45.],
       [ -52.],
       [ -49.],
       [ -49.],
       [ -36.],
       [ -23.],
       [ -20.],
       [ -29.],
       [ -70.],
       [ -92.],
       [ -86.],
       [-115.],
       [-196.]])